#### 功能说明：
+ 遍历原始数据，生成训练集txt文件和测试集txt文件
+ txt文件中一行为一个图片的数据，数据以空格分割，第一个元素是图片路径，后面元素依次是每一个物体的xmin，ymin，xmax和ymax

#### 原数据格式要求：
+ 数据存放在data_cmdt指向的文件夹中，每次收集的数据以日期命名为一个文件夹
+ 一次数据文件夹内分类编号命名的文件夹，品类数用N表示，其中0到N-1为单独分类的物品，N及N以上为混合拍摄的物品
+ 每个分类文件夹内，包含0-4命名的共5个文件夹，为相应摄像头的图像数据，此外还包括0.xml-4.xml共5个文件，为相应的标注文件
+ 对0到N-1文件夹内的数据，标注的分类包括good，null和bar
+ 对N及N以上的文件夹内的数据，标注的分类为分类编号（0到N-1的数字）
+ 要求进入到这个文件夹的数据，顶部摄像头的图像都必须是裁剪过的，顶部摄像头图像对应的标注文件，都必须是针对裁剪过的图像进行标注的文件

In [ ]:
import os
import xml.etree.ElementTree as ET 
import struct
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.core.debugger import Tracer